In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the data
df = pd.read_csv("./fraudTrain.csv")
df['gender'].replace(['F','M'],[0,1],inplace=True)
selected_features = ['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long','gender']

# Extract selected features and target variable
X = df[selected_features]
y = df['is_fraud']

# # data.head()
# # Drop unnecessary columns or those requiring extensive processing for simplicity
# drop_columns = ['first', 'last', 'street', 'city', 'trans_num', 'dob', 'trans_date_trans_time']
# data = data.drop(columns=drop_columns)

# # Encode categorical features
# categorical_columns = ['merchant', 'category', 'gender', 'state', 'job']
# data = pd.get_dummies(data, columns=categorical_columns)

# # Convert 'unix_time' to datetime
# data['transaction_datetime'] = pd.to_datetime(data['unix_time'], unit='s')

# # Extract relevant features from datetime
# data['transaction_day'] = data['transaction_datetime'].dt.day

# # Drop the original 'unix_time' and 'transaction_datetime' columns
# data = data.drop(columns=['unix_time', 'transaction_datetime'])

# # Standardize numerical features
# numerical_columns = ['cc_num', 'zip', 'lat', 'long', 'city_pop', 'amt', 'merch_lat', 'merch_long']
# scaler = StandardScaler()
# data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# # # Split the data into features (X) and target variable (y)
# # X = data.drop(columns=['is_fraud'])
# # y = data['is_fraud']
X.columns

Index(['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'merch_lat',
       'merch_long', 'gender'],
      dtype='object')

In [38]:
# import pandas as pd
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# # Assuming df is your DataFrame containing the data

# # Drop the 'Unnamed: 0' column if it's not needed
# df = df.drop(columns=['Unnamed: 0'])

# # Encode categorical features
# categorical_columns = ['merchant', 'category', 'gender', 'state', 'job']
# df_encoded = pd.get_dummies(df, columns=categorical_columns)

# # Encode datetime features
# df_encoded['trans_date_trans_time'] = pd.to_datetime(df_encoded['trans_date_trans_time'])
# df_encoded['dob'] = pd.to_datetime(df_encoded['dob'])

# # Extract relevant features from datetime
# df_encoded['trans_date'] = df_encoded['trans_date_trans_time'].dt.date
# df_encoded['trans_time'] = df_encoded['trans_date_trans_time'].dt.time

# # Drop the original datetime columns
# df_encoded = df_encoded.drop(columns=['trans_date_trans_time'])

# # Print the encoded DataFrame
# print("Encoded DataFrame:\n", df_encoded)

In [39]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Assuming X_train, X_test, y_train, y_test are your feature and target variable splits

# Apply RandomUnderSampler to undersample the majority class
undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)
print(y_resampled.value_counts())




is_fraud
0    7506
1    7506
Name: count, dtype: int64


In [40]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=5)  # Choose the number of components you want
# X_pca = pca.fit_transform(X_resampled)


In [41]:
# X_pca.shape

In [42]:
from sklearn.utils import shuffle

# Combine X and y into a single DataFrame
data_combined = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)],axis=1)

#shuffle data
shuffled_data = shuffle(data_combined, random_state=42)
shuffled_data.tail()
# Randomly sample 300 points from the combined data
sampled_data = shuffled_data.sample(n=500, random_state=42)

# # Separate the sampled data back into X and y
X_new = sampled_data.drop(columns=['is_fraud'])
y_new = sampled_data['is_fraud']
y_new.value_counts()

is_fraud
1    263
0    237
Name: count, dtype: int64

In [43]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

In [44]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [45]:
y_train.value_counts()

is_fraud
1    208
0    192
Name: count, dtype: int64

In [46]:
# from imblearn.under_sampling import RandomUnderSampler
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.metrics import classification_report

# # Assuming X_train, X_test, y_train, y_test are your feature and target variable splits

# # Apply RandomUnderSampler to undersample the majority class
# undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
# X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)
# X_test_resampled, y_test_resampled = undersampler.fit_resample(X_test, y_test)
# print(y_train_resampled.value_counts())
# print(y_test_resampled.value_counts())



In [47]:
# Train SVM on the resampled data
# svm_model = SVC(kernel='linear', random_state=42)
svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test_scaled)

# Evaluate the model
classification_report_result = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_report_result)

Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.96      0.78        45
           1       0.94      0.60      0.73        55

    accuracy                           0.76       100
   macro avg       0.80      0.78      0.76       100
weighted avg       0.82      0.76      0.76       100



In [48]:
from sklearn.metrics import classification_report, confusion_matrix

confusion_matrix_result = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion_matrix_result)

Confusion Matrix:
 [[43  2]
 [22 33]]


## Quantum Model

In [49]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, BasicAer, IBMQ, execute, Aer, assemble
# from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import Aer
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer



from qiskit.providers.aer import QasmSimulator, StatevectorSimulator, UnitarySimulator


# General libraries\
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import cm
from sklearn import metrics

# QKT related
from qiskit.circuit.library import ZZFeatureMap,ZFeatureMap, PauliFeatureMap
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel, TrainableFidelityQuantumKernel,QuantumKernel
# from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit.algorithms.state_fidelities import ComputeUncompute
# from qiskit.primitives import Sampler

# Additional imports
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from collections import Counter

In [50]:
data_map = lambda x: x[0]*x[0]+1
components=9
# components =17
fm = ZFeatureMap(components, reps=1,data_map_func=data_map)
# print(circuit_drawer(fm, output='text'))

# fm = ZZFeatureMap(components, entanglement='circular')
print(transpile(fm,optimization_level=3).depth())

2


In [51]:
# Create a quantum kernel using the feature map
quantum_kernel = QuantumKernel(feature_map=fm, quantum_instance=Aer.get_backend('qasm_simulator'))
# quantum_kernel = TrainableFidelityQuantumKernel(feature_map=fm, training_parameters=training_params)

# Create a QSVC model
qsvc = QSVC(quantum_kernel=quantum_kernel)

# Fit the QSVC model using the training data
qsvc.fit(X_train_scaled, y_train)

QSVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', max_iter=-1,
     probability=False,
     quantum_kernel=<qiskit_machine_learning.kernels.quantum_kernel.QuantumKernel object at 0x7f5118c20be0>,
     random_state=None, shrinking=True, tol=0.001, verbose=False)

In [52]:
# Predict the labels
labels_test = qsvc.predict(X_test_scaled)



In [53]:
# Evalaute the test accuracy
accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=labels_test)
print(confusion_matrix(y_test, labels_test))
print(classification_report(y_test, labels_test))
print(f"Test Accuracy: {accuracy_test}")

[[34 11]
 [26 29]]
              precision    recall  f1-score   support

           0       0.57      0.76      0.65        45
           1       0.72      0.53      0.61        55

    accuracy                           0.63       100
   macro avg       0.65      0.64      0.63       100
weighted avg       0.65      0.63      0.63       100

Test Accuracy: 0.6414141414141414


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming your data is already preprocessed as described in the previous response

# # Split the data into features (X) and target variable (y)
# X = data.drop(columns=['is_fraud'])
# y = data['is_fraud']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Standardize numerical features
# scaler = StandardScaler()
# X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
# X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

# Initialize the SVM model
svm_model = SVC(kernel='linear', random_state=42)

# Train the SVM model
svm_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report_result)
